In [ ]:
ALPHABET_SIZE = 256

def bm(pat, txt):
    sizeP = len(pat)
    sizeT = len(txt)
    ocurrences = []

    # --- 0. SUFFIXES TABLE ---
    suff = [0 for i in range(sizeP)]
    suff[sizeP-1] = sizeP
    g = sizeP - 1
    f = 0
    for i in range(sizeP-2, -1, -1):
        if i > g and suff[i + sizeP - 1 - f] < i-g:
            suff[i] = suff[i + sizeP - 1 - f]
        else:
            if i < g:
                g = i
            f = i
            while g >= 0 and pat[g] == pat[g + sizeP - 1 - f]:
                g -= 1
            suff[i] = f-g

    # --- 1. BAD CHARACTER TABLE ---
    bcT = []
    # This part is changed on the instrumentation section. Here we use List Comprehension.
    bcT = [sizeP for i in range(ALPHABET_SIZE)]
    for j in range(sizeP):
        bcT[ord(pat[j])] = sizeP - j

    # --- 2. GOOD SUFFIX TABLE ---
    gsT = []
    gsT = [sizeP for i in range(sizeP)]
    for i in range(sizeP-1, -1, -1):
        if suff[i] == i+1:
            for j in range(0, sizeP-1-i):
                if gsT[j] == sizeP:
                    gsT[j] = sizeP - 1 - i
    for i in range(0, sizeP-1):
        gsT[sizeP-1-suff[i]] = sizeP - 1 - i

    # --- 3. SEARCH STAGE ---
    for i in range(0, sizeT-sizeP+1):
        submatch = 0
        for j in range(sizeP-1, -1, -1):
            if(pat[j] != txt[i+j]):
                break
            submatch += 1
        if submatch == sizeP:
            ocurrences.append(f"Ocurrence at index {i}!")
        else:
            i += max(bcT[ord(txt[i+j])], gsT[j])
    return ocurrences
# https://www-igm.univ-mlv.fr/~lecroq/string/node14.html
# https://www.cs.utexas.edu/users/moore/publications/fstrpos.pdf 
# http://pages.di.unipi.it/pisanti/DIDATTICA/patternmatching2.pdf 

In [ ]:
ALPHABET_SIZE = 256


def bm_instrumentado(pat, txt):
    sizeP = len(pat)
    sizeT = len(txt)
    ocurrences = []
    cont = 0

    # NO CONTADA EN LA INSTRUMENTACION
    # --- 0. SUFFIXES TABLE ---
    suff = [0 for i in range(sizeP)]
    suff[sizeP-1] = sizeP
    g = sizeP - 1
    f = 0
    for i in range(sizeP-2, -1, -1):
        if i > g and suff[i + sizeP - 1 - f] < i-g:
            suff[i] = suff[i + sizeP - 1 - f]
        else:
            if i < g:
                g = i
            f = i
            while g >= 0 and pat[g] == pat[g + sizeP - 1 - f]:
                g -= 1
            suff[i] = f-g

    # --- 1. BAD CHARACTER TABLE ---
    bcT = []
    # This part is changed on the instrumentation section. Here we use List Comprehension.
    bcT = []
    for i in range(ALPHABET_SIZE):
        cont += 1
        bcT.append(sizeP)
    for j in range(sizeP):
        cont += 1
        bcT[ord(pat[j])] = sizeP - j

    # --- 2. GOOD SUFFIX TABLE ---
    gsT = []
    gsT = [sizeP for i in range(sizeP)]
    for i in range(sizeP-1, -1, -1):
        if suff[i] == i+1:
            for j in range(0, sizeP-1-i):
                cont += 1
                if gsT[j] == sizeP:
                    gsT[j] = sizeP - 1 - i
    for i in range(0, sizeP-1):
        cont += 1
        gsT[sizeP-1-suff[i]] = sizeP - 1 - i

    # --- 3. SEARCH STAGE ---
    for i in range(0, sizeT-sizeP+1):
        submatch = 0
        for j in range(sizeP-1, -1, -1):
            cont += 1
            if(pat[j] != txt[i+j]):
                break
            submatch += 1
        if submatch == sizeP:
            ocurrences.append(f"Ocurrence at index {i}!")
        else:
            i += max(bcT[ord(txt[i+j])], gsT[j])
    return cont


print(bm_instrumentado('aa', 'aaaa'))

def test_BM_instrumentado(filename, start, end, inc):
    graph = [[], [], [], []]
    for i in range(start, end, inc):
        txt = "a" * i
        pat = "aaa"

        graph[0].append(len(txt) * len(pat))
        graph[1].append(bm_instrumentada(pat, txt))

        txt2 = "a" * 300
        pat2 = "a" * i
        graph[2].append(len(txt2) * len(pat2))
        graph[3].append(bm_instrumentada(pat2, txt2))
    return graph


g = test_BM_instrumentado('BM_instrumentado.csv', 5, 300, 5)
tempX = list(range(300))
plt.plot(g[0], g[1], color="blue", label="BM(pat constante)", linewidth=2.0)
plt.plot(g[2], g[3], '--', color="red", label="BM(txt constante)")
plt.plot(tempX, tempX, '--', color='yellow', label="identidad")
plt.legend()
plt.xlabel('Time')
plt.ylabel("N")
plt.title("Analisis de Tiempo de BM")
plt.show()